In [110]:
import pandas as pd
#import matplotlib.pyplot as plt
import numpy as np
#from fancyimpute import KNN
#from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, mean_squared_error
from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier, XGBRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
#from sklearn.preprocessing import LabelEncoder
#from sklearn.model_selection import cross_val_score

#from ggplot import *

In [104]:
df = pd.read_csv("../data/WNS/train.csv")
df.columns = df.columns.str.replace("[?>% ]","")
df.head()
#temporarily fill na
df.education.fillna(-1,inplace=True) #"Bachelor's")
df.previous_year_rating.fillna(-1,inplace=True) #3
df.isnull().sum()
ohe = ['department', 'region', 'education', 'gender', 'recruitment_channel']
dfnew = pd.get_dummies(data=df)
#(len(df) - (df.isnull().sum()))/len(df)
#df.apply(lambda x:x.isnull().sum())
#df.shape

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met80,awards_won,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [111]:
y = dfnew.is_promoted
x_train, x_test, y_train, y_test = train_test_split(dfnew.drop(['is_promoted'],axis=1),y,test_size=0.4,random_state=42)
train_idx = x_train.employee_id
test_idx = x_test.employee_id
x_train = x_train.drop(['employee_id'],axis=1)
x_test = x_test.drop(['employee_id'],axis=1)


In [134]:
#rem = ['employee_id', 'is_promoted']
#X = dfnew.drop(rem,axis=1)
#y = dfnew.is_promoted
pred = pd.DataFrame()
for i in range(5):
    xgb = XGBRegressor(learning_rate=0.1,max_depth=7,n_estimators=100)
    clf_xgb = xgb.fit(X=x_train,y=y_train)
    p=clf_xgb.predict(x_test)
    pred=pd.concat([pred,pd.DataFrame(p)],axis=1)

In [138]:
c=pred.mean(axis=1)
c[c<=0.25] = 0
c[c>0.25] = 1
f1_score(y_true=y_test,y_pred=c)

0.511875

In [91]:
#df.loc[:,['gender','is_promoted']].groupby(['is_promoted']).describe()
#looks like only aprox 10% ppl gets the promotion and female and male almost equally get a chance
#**promotion is not biased by gender
#df.groupby(['education','is_promoted']).size()
#df.groupby(['previous_year_rating','is_promoted']).size()y

In [305]:
#check the prob of a person's education level given gender, age, department, avg_training_score 
#and then impute the education NaN values based on this prob by sampling

In [74]:
#clf = RandomForestClassifier() #-0.36-0.37
#clf = LogisticRegression() ## 0.384 without FE

#train on full data


In [139]:
## test
sub = pd.read_csv('../data/WNS/test.csv')
sub.columns = sub.columns.str.replace('[?>% ]','')
#temporarily fill na
sub.education.fillna(-1,inplace=True) #"Bachelor's")
sub.previous_year_rating.fillna(-1,inplace=True) #3

ohe = ['department', 'region', 'education', 'gender', 'recruitment_channel']
subnew = pd.get_dummies(data=sub)

In [116]:
#temporarily fill na
#sub.education=sub.education.fillna("Bachelor's")
#sub.previous_year_rating = sub.previous_year_rating.fillna(int(3))
#sub.isnull().sum()
#subnew=pd.get_dummies(sub)
subnew.describe()

,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met80,awards_won,avg_training_score,department_Analytics,department_Finance,...,region_region_9,education_-1,education_Bachelor's,education_Below Secondary,education_Master's & above,gender_f,gender_m,recruitment_channel_other,recruitment_channel_referred,recruitment_channel_sourcing
count,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,...,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000
mean,39041.399149,1.254236,34.782929,3.004427,5.810387,0.358834,0.022776,63.263133,0.098723,0.046445,...,0.007663,0.044019,0.663176,0.015922,0.276884,0.293487,0.706513,0.556748,0.019200,0.424053
std,22640.809201,0.600910,7.679492,1.677263,4.207917,0.479668,0.149191,13.411750,0.298296,0.210452,...,0.087203,0.205141,0.472634,0.125175,0.447468,0.455369,0.455369,0.496780,0.137229,0.494209
min,3.000000,1.000000,20.000000,-1.000000,1.000000,0.000000,0.000000,39.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19370.250000,1.000000,29.000000,2.000000,3.000000,0.000000,0.000000,51.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,38963.500000,1.000000,33.000000,3.000000,5.000000,0.000000,0.000000,60.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
75%,58690.000000,1.000000,39.000000,4.000000,7.000000,1.000000,0.000000,76.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,78295.000000,9.000000,60.000000,5.000000,34.000000,1.000000,1.000000,99.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [140]:
xgbsub = clf_xgb.predict(subnew.drop(['employee_id'],axis=1))
xgbsub[xgbsub<=0.25] = 0
xgbsub[xgbsub>0.25] = 1

In [143]:
submission = pd.concat([subnew.employee_id,pd.DataFrame(xgbsub)],axis=1)

In [144]:
submission.columns = ['employee_id','is_promoted']
#submission.groupby(['is_promoted']).size()
submission.to_csv('../data/WNS/wns_xgb_submission3.csv',index=False)
submission

,employee_id,is_promoted
0,8724,0.0
1,74430,0.0
2,72255,0.0
3,38562,0.0
4,64486,0.0
5,46232,0.0
6,54542,0.0
7,67269,0.0
8,66174,0.0
9,76303,0.0
